In [1]:
#Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install googledrivedownloader

In [4]:
!cp /content/drive/MyDrive/dldataset.zip .
!unzip dldataset.zip -d extracted_data/

Archive:  dldataset.zip
   creating: extracted_data/dldataset/
  inflating: extracted_data/dldataset/01_02__outside_talking_still_laughing__YVGY8LOK.mp4  
  inflating: extracted_data/dldataset/01_02__walk_down_hall_angry__YVGY8LOK.mp4  
  inflating: extracted_data/dldataset/01_03__hugging_happy__ISF9SP4G.mp4  
  inflating: extracted_data/dldataset/01__exit_phone_room.mp4  
  inflating: extracted_data/dldataset/01__hugging_happy.mp4  
  inflating: extracted_data/dldataset/01__kitchen_pan.mp4  


In [5]:
#To get the average frame count
import json
import glob
import numpy as np
import cv2
import copy

video_files =  glob.glob('/content/extracted_data/dldataset/*.mp4')
#video_files1 =  glob.glob('/content/dfdc_train_part_0/*.mp4')
#video_files += video_files1
frame_count = []
for video_file in video_files:
  cap = cv2.VideoCapture(video_file)
  if(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))<150):
    video_files.remove(video_file)
    continue
  frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
print("frames" , frame_count)
print("Total number of videos: " , len(frame_count))
print('Average frame per video:',np.mean(frame_count))

frames [305, 605, 560, 787, 727, 217]
Total number of videos:  6
Average frame per video: 533.5


In [7]:
!pip3 install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566162 sha256=3d19f9174104dcf0b8bfd51f2cc95ec2ec5cdde6434d2c13d3d0bc1187cd30f3
  Stored in directory: /root/.cache/pip/wheels/04/52/ec/9355da79c29f160b038a20c784db2803c2f9fa2c8a462c176a
Successfully built face-recognition-models


In [8]:
# to extract frame
def frame_extract(path):
  vidObj = cv2.VideoCapture(path)
  success = 1
  while success:
      success, image = vidObj.read()
      if success:
          yield image

In [9]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import face_recognition
from tqdm.autonotebook import tqdm

<ipython-input-9-2563df55c1b7>:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [16]:
!mkdir '/content/Face_only_data'

In [17]:
# process the frames
def create_face_videos(path_list,out_dir):
  already_present_count =  glob.glob(out_dir+'*.mp4')
  print("No of videos already present " , len(already_present_count))
  for path in tqdm(path_list):
    out_path = os.path.join(out_dir,path.split('/')[-1])
    file_exists = glob.glob(out_path)
    if(len(file_exists) != 0):
      print("File Already exists: " , out_path)
      continue
    frames = []
    flag = 0
    face_all = []
    frames1 = []
    out = cv2.VideoWriter(out_path,cv2.VideoWriter_fourcc('M','J','P','G'), 30, (112,112))
    for idx,frame in enumerate(frame_extract(path)):
      #if(idx % 3 == 0):
      if(idx <= 150):
        frames.append(frame)
        if(len(frames) == 4):
          #faces = face_recognition.batch_face_locations(frames, model="hog")
          faces = [face_recognition.face_locations(frame) for frame in frames]
          #faces = face_recognition.batch_face_locations(frames)
          for i,face in enumerate(faces):
            if(len(face) != 0):
              top,right,bottom,left = face[0]
            try:
              out.write(cv2.resize(frames[i][top:bottom,left:right,:],(112,112)))
            except:
              pass
          frames = []
    try:
      del top,right,bottom,left
    except:
      pass
    out.release()

In [18]:
print("CUDA Available:", torch.cuda.is_available())

CUDA Available: True


In [19]:
create_face_videos(video_files,'/content/Face_only_data')

No of videos already present  0


  0%|          | 0/6 [00:00<?, ?it/s]